In [1]:
import flowio
import numpy as np
import pandas as pd
import os

In [3]:
file_path = 'OriginalDATA/FlowRepository_FR-FCM-Z24N_files'
files = os.listdir(file_path)
files.sort()

In [5]:
controls = [f for f in files if "KTR" in f]

unstim_control = [f for f in controls if 'unstim' in f]
stim_control = [f for f in controls if ' stim' in f]

In [7]:
ras = [f for f in files if "RA" in f]

unstim_ras = [f for f in ras if "unstim" in f]
stim_ras = [f for f in ras if " stim" in f]

### TODO
- get a list of the channels that I want with their channel number
- from the pandas get this channels
- name the channels appropriately

### Note
- Cannot find functional marker p-Erk

# RA-data description

Mass cytometry (CyTOF)
1. panel of 12 phenotyping and 10 functional markers
2. signaling in unstimulated and TNF-stimulated peripheral blood mononulear cells
3. from 20 newly diagnosed, untreated RA patients
4. 20 healthy donors.

1. unstim = unstimulated with tumor necrosis factor (TNF) (TNF inhibitors effectively repress inflammatory activity in RA)
2. stim = stimulated with TNF
3. ungated = manual removal of events representing 

## The RA paper:
The objective of this study was to identify markers in immune cell populations that distinguish RA patients from healthy donors with an emphasis on TNF signaling

We employed mass cytometry (CyTOF) with a panel of 13 phenotyping and 10 functional markers to explore signaling in unstimulated and TNF-stimulated peripheral blood mononuclear cells from 20 newly diagnosed, untreated RA patients and 20 healthy donors.

1. intracellular functional markers
2. surface phenotypic markers 

problem:
- want to collect more cells for each patient
    - don't know what file belongs to each patient
    - don't know which of the data is preprosessed or not
- want to use more markers <b style="color:green"> DONE </b> 
    - don't know how to get the other markers 
    - don't know which markers are phenotypical and which are functional
    

In [143]:
def get_events(file, markers, nr_events):
    f = flowio.flowdata.FlowData(file)
    marker_channels = dict()
    for j in markers:
        for k in f.channels.keys():
            if (j in f.channels[k]['PnS']):
                marker_channels[int(k)] = f.channels[k]['PnS']
    
    npy_data = np.reshape(f.events, (-1, f.channel_count))
    subsample = random.sample(list(npy_data), nr_events)
    df = pd.DataFrame(subsample)
    df2 = df.rename(columns=marker_channels)
    return df2[list(marker_channels.values())]

In [14]:
# choose files
# choose nr of cells per patient
# concatenate all files into one df
# save the file
# write about the data

In [235]:
def get_patients(group_files, nr_cells, file_path, markers, group_name, id_key):
    all_df = None
    for i, file in enumerate(group_files):
        path = file_path + "/" + file
        df = get_events(path, markers, nr_cells)
        #key = "({}-[0-9]+)".format(id_key)
        df["id"] = i #re.findall(key, file)[0]
        if i == 0:
            all_df = df
        else:
            all_df = all_df.append(df, ignore_index=True)
    all_df["group"] = group_name
    return all_df

phenotyping for clustering : cell type

functional for within clustering : what happens IN the cells

In [13]:
# Taken from the paper
phenotyping = ['CD20','CD3','CD4','CD8a','CD45RA', 'CD56', 
               'CD16', 'CD14', 'CD61', 'CD11c','CD123', 'HLA-DR', 'CD45']

functional = ['Caspase3', 'CD86','p-p38','p-Erk','p-Akt','p-cJun','p-NFkB','IkBa','CD120a','CD120b']
# missing one funcitonal, p-Erk. The metal expected 171Yb is not used attached to any antibody. This is explained in the paper

In [21]:
def scale_df(df):
    df_sub = df[df.columns.difference(["id","group"])]
    raw_scaled_df = df_sub.transform(lambda x: np.arcsinh(x/5))
    df_scaled = raw_scaled_df.copy()
    df_scaled["id"] = df.id
    df_scaled["group"] = df.group
    return df_scaled

In [23]:
df = scale_df(df)

In [252]:
df

,142Nd_Caspase3,145Nd_CD4,146Nd_CD8a,147Sm_CD20,148Nd_CD16,150Nd_CD86,151Eu_CD123,152Sm_p-Akt,155Gd_CD120a,156Gd_p-p38,...,166Er_p-NFkB,167Er_p-cJun,169Tm_CD45RA,170Er_CD3,174Yb_HLA-DR,176Yb_CD56,209Bi_CD61,89Y_CD45,id,group
3249703,0.079716,4.862322,0.000000,0.425815,0.0,0.258898,0.382597,0.081510,0.837722,0.0,...,0.000000,0.114152,4.010632,0.0,0.0,0.0,5.528366,0.0,0,diseased
3196090,0.000000,0.000000,0.000000,0.000000,0.0,0.354715,0.000000,0.000000,0.886878,0.0,...,0.434590,0.000000,0.000000,0.0,0.0,0.0,5.846364,0.0,0,diseased
67638,0.000000,2.514892,0.077323,0.000000,0.0,0.000000,0.165642,0.000000,0.447694,0.0,...,0.671731,0.000000,0.000000,0.0,0.0,0.0,5.852854,0.0,0,control
3251092,0.092866,5.083666,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,3.827497,0.0,0.0,0.0,5.740776,0.0,0,diseased
3290766,0.000000,1.059655,0.000000,0.000000,0.0,1.012897,0.322767,0.000000,3.693521,0.0,...,2.283759,0.000000,1.689329,0.0,0.0,0.0,5.697975,0.0,0,diseased
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6259500,0.000000,0.799921,0.000000,0.000000,0.0,0.227629,0.000000,0.437691,1.240473,0.0,...,0.332632,0.000000,1.005044,0.0,0.0,0.0,5.923002,0.0,19,diseased
6240094,0.000000,1.244736,0.000000,0.000000,0.0,0.000000,0.484251,0.000000,0.000000,0.0,...,0.668000,0.473148,4.522711,0.0,0.0,0.0,5.572564,0.0,19,diseased
3131693,0.000000,0.013000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.968520,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,5.766790,0.0,19,control
3077071,0.000000,0.875279,0.000000,0.000000,0.0,0.000000,0.389478,0.000000,0.716344,0.0,...,1.110713,0.000000,0.000000,0.0,0.0,0.0,5.824301,0.0,19,control


In [29]:
df.to_csv("ModifiedDATA/scaled_ra2.csv", index=False)

In [31]:
df.group.unique()

array(['control'], dtype=object)

TODO
- prepare the 40 patients with the same number of events, and find such maximum item. (unstim)
    - phenotyping
    - functional
    - phenotyping and functional
- prepare the 40 patients with the same number of events, and find such maximum item. (stim for later)
    - phenotyping
    - functional
    - phenotyping and functional
- scale the values

In [54]:
# find the minimum nr of cells in all files

In [59]:
minimum_control = float("inf")
for file in controls:
    f = flowio.flowdata.FlowData(file_path + "/" + file)
    if f.event_count < minimum_control:
        minimum_control = f.event_count

In [60]:
minimum_ras = float("inf")
for file in ras:
    f = flowio.flowdata.FlowData(file_path + "/" + file)
    if f.event_count < minimum_ras:
        minimum_ras = f.event_count

In [155]:
nr_cells

159736

In [167]:
#nr_cells = 20000
nr_cells = min(minimum_control, minimum_ras) # this gives 3194720 events per group, 6389440 in total for certain datatype.




In [ ]:
# max nr cells = 159736, i.e. n = 79868
# n = 20 000, this might be too little.
# n = 40 000
# n = 50 000
# n = 75 000 this the maximum we can get.

data_shape = {"ideal_ddloss" : (n, n),
              "train + dd_test": (n*0.8, n*0.2), 
              "dimred_train + rest": (n*0.2*0.8, n*0.8**2), 
              "model testing" : (n*0.8**2, n*0.8*0.2), 
              "model validation" : (n*0.8**3, n*0.8**2*0.2) # if k-fold is used, then this is not constant
             }

### Notes
- possible issues with such large data set?
    - need large models, takes longer to train
    - takes time with dimensionality reduction
- Advantage:
    - can train better models, and avoid variance in the model
- comparison:
    - MNIST is 21 MB sized file
    - Extended MNIST is 535 MB
    - Kuzushiji MNIST is 31 MB
    - BINARIZED MNIST is 104 MB
- To try, nr_cells per patient
    - 20 000
    - 50 000
    - 100 000
    - 150 000

In [236]:
%%time
#df_cp = get_patients(unstim_control, nr_cells, file_path, phenotyping, "control", "KTR") 
#df_rp = get_patients(unstim_ras, nr_cells, file_path, phenotyping, "diseased", "RA")

#df_cf = get_patients(unstim_control, nr_cells, file_path, functional, "control", "KTR")
#df_rf = get_patients(unstim_ras, nr_cells, file_path, functional, "diseased", "RA")

df_cpf = get_patients(unstim_control, nr_cells, file_path, phenotyping + functional, "control", "KTR")
df_rpf = get_patients(unstim_ras, nr_cells, file_path, phenotyping + functional, "diseased", "RA")


CPU times: user 3min 8s, sys: 18.5 s, total: 3min 26s
Wall time: 3min 29s


In [258]:
df_pf.group.unique()

array(['control', 'diseased'], dtype=object)

In [240]:
#df_p = df_cp.append(df_rp)
#df_f = df_cf.append(df_rf)
df_pf = df_cpf.append(df_rpf, ignore_index=True)


In [241]:
#df_p = scale_df(df_p)
#df_f = scale_df(df_f)
df_pf = scale_df(df_pf)


In [255]:
#df_p.to_csv("ModifiedDATA/RA/Scaled/unstim_pheno.csv", index=False)
#df_f.to_csv("ModifiedDATA/RA/Scaled/unstim_func.csv", index=False)
df_pf.to_csv("ModifiedDATA/RA/Scaled/unstim_phenofunc.csv", index=False)


### TODO
- split the data
    - sample 40000 per patient
    - pheno / func
    - train, val, test1, test2, Ideal

In [205]:
from sklearn.model_selection import train_test_split

In [243]:
n1, n2 = train_test_split(df_cpf, test_size=0.5, random_state=42, stratify=df_cpf['id'])

In [244]:
df_cpf.loc[n1.index[:2]]

,147Sm_CD20,170Er_CD3,89Y_CD45,145Nd_CD4,169Tm_CD45RA,146Nd_CD8a,176Yb_CD56,148Nd_CD16,160Gd_CD14,209Bi_CD61,...,150Nd_CD86,156Gd_p-p38,152Sm_p-Akt,167Er_p-cJun,166Er_p-NFkB,164Dy_IkBa,155Gd_CD120a,165Ho_CD120b,id,group
2359342,0.000,0.0,0.0,0.00000,357.369995,0.0,0.0,0.0,0.0,710.083984,...,0.0,0.0,0.0,1.687,2.752,0.0,0.700,9.730000,14,control
1612764,0.962,0.0,0.0,261.86499,200.054001,0.0,0.0,0.0,0.0,700.236023,...,0.0,0.0,0.0,0.216,2.876,0.0,4.294,28.261999,10,control


In [247]:
%%time

# Load data

df_pf = pd.read_csv("ModifiedDATA/RA/Scaled/unstim_phenofunc.csv")

# subsample data

SUBSAMPLE_SIZE = 40000 # nr_cells per patient

df = df_pf.groupby('id', group_keys=False).sample(n=SUBSAMPLE_SIZE, replace=False, random_state=2)


# split size

RANDOM_SEED = 42

train_dd, test_dd = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED, stratify=df['id'])

trian_m , test = train_test_split(train_dd, test_size=0.2, random_state=RANDOM_SEED, stratify=train_dd['id'])

train, val = train_test_split(trian_m, test_size=0.2, random_state=RANDOM_SEED, stratify=trian_m['id'])

data = { "train": train,
         "val": val,
         "test": test,
         "test_dd":test_dd
       }

# use val + test for training dim_red
# use train + val for model selection based on density estimation
# use test for model selection based of dd-loss
# use test_dd for estimating the double date loss for the "ideal model".

# get the markers

functional_columns = list(filter((lambda x : any([i in x for i in functional])), df_pf.columns))
phenotyping_columns = list(filter((lambda x : any([i in x for i in phenotyping])), df_pf.columns))
labels = ["id", "group"]

#df_p = data["train"][phenotyping_columns + labels]
#df_f = data["train"][funcitonal_columns + labels]



CPU times: user 14.8 s, sys: 1.92 s, total: 16.8 s
Wall time: 16.9 s
